<a href="https://colab.research.google.com/github/calemhoffman/softhub/blob/master/python/mona_proposals.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Initialization

In [1]:
import plotly.express as px
import plotly.graph_objects as go
from plotly.graph_objs import Scatter
from plotly.subplots import make_subplots
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import plotly.io as pio
import math
#pio.templates

# 30F -> 29F

## Notes

In [2]:
# kevin fossez - Z=9 chain nov. 2020
# rebeka - fsu int, z=9 chain nov/dec 2020
# utsuno - sdpf-m, EEdf1 - Dec. 18, 2020
# fortuna - anything on halo nuclei Dec. 18, 2020
# ragnar - any calcs of interest Dec. 18, 2020
# doing sdpf-u locally (0p0h, mix, 2p2h)

## Input data

In [11]:
##@title FSU INT Data #j, energy, number, p-h
f29_fsu_data = np.array([[5/2,-208.378,1,2],
                     [1/2,-208.573,1,2],  
                     [9/2,-207.143,1,2],
                     [5/2,-206.938,2,0],
                     [3/2,-206.374,1,2],
                     [1/2,-203.436,2,0]])

f30_fsu_data = np.array([[2,-206.573,1,2],
                         [3,-206.428,1,2],
                         [4,-206.224,1,2],
                         [4,-205.930,2,0],
                         [2,-205.899,2,0],
                         [6,-205.278,1,0],
                         [6,-205.806,2,2]])

f_binding = pd.DataFrame({'interaction': ['fsu','sdpf-m','gsm','gsm-fht','gsm-eft','hf-mbvt','sdpf-u'],
                      'f25_be': [0.0,-25.238,-21.674,-14.437,-14.350,-14.437,0.0],
                      'f26_be': [-204.920,-25.473,-22.809,-15.531,-15.280,-15.367,0.0],
                      'f27_be': [-206.953,-28.136,-24.533,-17.610,-16.975,-16.943,0.0],
                      'f28_be': [-206.517,-27.548,-24.728,-17.347,-17.260,-17.140,0.0],
                      'f29_be': [-208.378,-29.115,-25.316,-18.770,-18.650,-18.343,-60.685],
                      'f30_be': [-206.573,-27.822,-22.104,-18.179,-18.540,-16.811,-59.916],
                      'f31_be': [-207.596,-28.841,-23.044,-18.803,-18.781,-17.315,-59.853],
                      'f32_be': [-205.887,-27.039,-15.251,0.0,0.0,0.0,0.0]})
f_binding

,interaction,f25_be,f26_be,f27_be,f28_be,f29_be,f30_be,f31_be,f32_be
0,fsu,0.000,-204.920,-206.953,-206.517,-208.378,-206.573,-207.596,-205.887
1,sdpf-m,-25.238,-25.473,-28.136,-27.548,-29.115,-27.822,-28.841,-27.039
2,gsm,-21.674,-22.809,-24.533,-24.728,-25.316,-22.104,-23.044,-15.251
3,gsm-fht,-14.437,-15.531,-17.610,-17.347,-18.770,-18.179,-18.803,0.000
4,gsm-eft,-14.350,-15.280,-16.975,-17.260,-18.650,-18.540,-18.781,0.000
5,hf-mbvt,-14.437,-15.367,-16.943,-17.140,-18.343,-16.811,-17.315,0.000
6,sdpf-u,0.000,0.000,0.000,0.000,-60.685,-59.916,-59.853,0.000


In [12]:
f29_fsu_df = pd.DataFrame({'j':f29_fsu_data[:,0], 'e':f29_fsu_data[:,1],
                       'n':f29_fsu_data[:,2], 'p-h':f29_fsu_data[:,3]})
f30_fsu_df = pd.DataFrame({'j':f30_fsu_data[:,0], 'e':f30_fsu_data[:,1],
                       'n':f30_fsu_data[:,2], 'p-h':f30_fsu_data[:,3]})
num_rows, num_cols = f29_fsu_data.shape
f29_x_data = np.zeros(num_rows)
num_rows, num_cols = f30_fsu_data.shape
f30_x_data = np.ones(num_rows)
# print(f30_fsu_df.iat[0,1])

In [92]:
#Experimental data
#be/a 
f_exp_be = pd.DataFrame({'f25_be' : [25.012167727,103.535E-6],
                         'f26_be' : [26.020020392,119.377E-6],
                         'f27_be' : [27.027322000,418.500E-6],
                         'f28_be' : [28.036223095,421.928E-6],
                         'f29_be' : [29.043103000,564.000E-6],
                         'f30_be' : [0,0],
                         'f31_be' : [0,0],
                         'f32_be' : [0,0]})

amu=931.494
n=1.00866491582
num_rows,num_cols = f_exp_be.shape
f_Sn_exp = pd.DataFrame(0.0,index=range(num_rows*2),columns=range(num_cols))
for i in range(num_cols-1):
  if f_exp_be.iat[0,i+1] != 0:
    #print((f_exp_be.iat[0,i+1] - f_exp_be.iat[0,i] - n)*amu)
    f_Sn_exp.iat[0,i+1] = (f_exp_be.iat[0,i+1] - f_exp_be.iat[0,i] - n)*amu
    f_Sn_exp.iat[1,i+1] = (math.sqrt(f_exp_be.iat[1,i+1]*f_exp_be.iat[1,i+1]+
                          f_exp_be.iat[1,i]*f_exp_be.iat[1,i]))*amu
    if f_exp_be.iat[0,i+2] != 0:
      #print((f_exp_be.iat[0,i+2] - f_exp_be.iat[0,i] - 2.0*n)*amu)
      f_Sn_exp.iat[2,i+2] = (f_exp_be.iat[0,i+2] - f_exp_be.iat[0,i] - 2.0*n)*amu
      f_Sn_exp.iat[3,i+2] = (math.sqrt(f_exp_be.iat[1,i+2]*f_exp_be.iat[1,i+2]+
                            f_exp_be.iat[1,i]*f_exp_be.iat[1,i]))*amu

f_Sn_exp = f_Sn_exp.T
f_Sn_exp = f_Sn_exp.rename(columns={0:'Sn',1:'Sn_err',2:'S2n',3:'S2n_err'})
f_Sn_exp = f_Sn_exp.drop([0,1])
f_Sn_exp = f_Sn_exp.rename(index={2:'<sup>27</sup>F',3:'<sup>28</sup>F',4:'<sup>29</sup>F',5:'<sup>30</sup>F',6:'<sup>31</sup>F'})
f_Sn_exp = f_Sn_exp*(-1.0)


## Plots

### Decay

In [29]:
markSize=100
f29_trace = go.Scatter(
    x=f29_x_data,           # x-coordinates of trace
    y=f29_fsu_df['e'],          # y-coordinates of trace
    #name='fsu int.',
    mode='markers + text',   # scatter mode
    marker = dict(size = markSize, symbol = 141), # Add symbol here!
     textfont=dict(color='black',size=18,family='Times New Roman')
    )

f30_trace = go.Scatter(
    x=f30_x_data,           # x-coordinates of trace
    y=f30_fsu_df['e'],          # y-coordinates of trace
    mode='markers + text',   # scatter mode
    marker = dict(size = markSize, symbol = 141), # Add symbol here!
     textfont=dict(color='black',size=18,family='Times New Roman')
    )
data = [f29_trace,f30_trace]

fig = go.Figure(data=data)

fig.update_layout(
    title='fsu interaction',
    width=600,
    height=300,
    margin=dict(l=35, r=35, t=35, b=35),
    #paper_bgcolor="LightSteelBlue",
    xaxis=dict(title='isotope',
               range=[-0.5,1.5],
               tickmode='array',tickvals=[0,1],ticktext=['<sup>29</sup>F','<sup>30</sup>F']
               ),
    yaxis=dict(title='binding energy [MeV]', range=[-209,-205]),
    showlegend=False,
    annotations=[dict(x=0.3, y=f29_fsu_data[0,1],ax=0.7, ay=f30_fsu_data[0,1],
                   xref="x",yref="y",axref="x",ayref="y",
                   showarrow=True,arrowhead=3,text=None),
                dict(x=0.3, y=f29_fsu_data[1,1],ax=0.7, ay=f30_fsu_data[0,1],
                   xref="x",yref="y",axref="x",ayref="y",
                   showarrow=True,arrowhead=3,text=None)],
                  template="plotly_dark"
    )

fig.show()

### Excite

In [30]:
markSize=100
f30_trace = go.Scatter(
    x=f30_x_data,           # x-coordinates of trace
    y=f30_fsu_df['e']-f30_fsu_df.iat[0,1]+0.01,          # y-coordinates of trace
    mode='markers + text',   # scatter mode
    marker = dict(size = markSize, symbol = 141), # Add symbol here!
     textfont=dict(color='black',size=18,family='Times New Roman'),
    #text=f30_fsu_df['e']-f30_fsu_df.iat[0,1]
    )
data = [f30_trace]
fig = go.Figure(data=data)
fig.update_layout(
    title='fsu interaction',
    width=300,
    height=300,
    margin=dict(l=35, r=35, t=35, b=35),
    #paper_bgcolor="LightSteelBlue",
    xaxis=dict(title='',
               range=[0.5,1.5],
               tickmode='array',tickvals=[1],ticktext=['<sup>30</sup>F'],
               showline=False,showgrid=False
               ),
    yaxis=dict(title='excitation energy [MeV]', range=[0,2]),
    showlegend=False,
    template="plotly_dark"
    )

fig.show()

### Sn systematics

In [96]:
f_Sn = pd.DataFrame()
f_Sn['<sup>27</sup>F'] = f_binding['f27_be']-f_binding['f26_be']
f_Sn['<sup>28</sup>F'] = f_binding['f28_be']-f_binding['f27_be']
f_Sn['<sup>29</sup>F'] = f_binding['f29_be']-f_binding['f28_be']
f_Sn['<sup>30</sup>F'] = f_binding['f30_be']-f_binding['f29_be']
f_Sn['<sup>31</sup>F'] = f_binding['f31_be']-f_binding['f30_be']
f_S2n = pd.DataFrame()
f_S2n['<sup>28</sup>F'] = f_binding['f28_be']-f_binding['f26_be']
f_S2n['<sup>29</sup>F'] = f_binding['f29_be']-f_binding['f27_be']
f_S2n['<sup>30</sup>F'] = f_binding['f30_be']-f_binding['f28_be']
f_S2n['<sup>31</sup>F'] = f_binding['f31_be']-f_binding['f29_be']
f_Sn = f_Sn.T
f_Sn = f_Sn.rename(columns={0: 'fsu', 1:'sdpf-m', 2:'gsm', 3:'gsm-fht', 4:'gsm-eft', 5:'hf-mbvt', 6:'sdpf-u'})
f_S2n = f_S2n.T
f_S2n = f_S2n.rename(columns={0: 'fsu', 1:'sdpf-m', 2:'gsm', 3:'gsm-fht', 4:'gsm-eft', 5:'hf-mbvt', 6:'sdpf-u'})
#imsrg
18, -29.452736318407954
19, -30.348258706467657
20, -30.54726368159204
21, -31.84079601990049
22, -31.84079601990049
23, -31.84079601990049
24, -32.338308457711435
#-30.945
f_Sn['imsrg'] = [-1.194,0.199,-0.796,1.293,0.00] #-1.493,-0.896
f_S2n['imsrg'] = [30.348-30.945,30.547-30.945,31.841-30.945,31.841-30.945]
print(f_S2n)
print(f_Sn)
f_Sn = f_Sn*(-1.0)
f_S2n = f_S2n*(-1.0)

                  fsu  sdpf-m    gsm  gsm-fht  gsm-eft  hf-mbvt  sdpf-u  imsrg
<sup>28</sup>F -1.597  -2.075 -1.919   -1.816   -1.980   -1.773   0.000 -0.597
<sup>29</sup>F -1.425  -0.979 -0.783   -1.160   -1.675   -1.400 -60.685 -0.398
<sup>30</sup>F -0.056  -0.274  2.624   -0.832   -1.280    0.329 -59.916  0.896
<sup>31</sup>F  0.782   0.274  2.272   -0.033   -0.131    1.028   0.832  0.896
                  fsu  sdpf-m    gsm  gsm-fht  gsm-eft  hf-mbvt  sdpf-u  imsrg
<sup>27</sup>F -2.033  -2.663 -1.724   -2.079   -1.695   -1.576   0.000 -1.194
<sup>28</sup>F  0.436   0.588 -0.195    0.263   -0.285   -0.197   0.000  0.199
<sup>29</sup>F -1.861  -1.567 -0.588   -1.423   -1.390   -1.203 -60.685 -0.796
<sup>30</sup>F  1.805   1.293  3.212    0.591    0.110    1.532   0.769  1.293
<sup>31</sup>F -1.023  -1.019 -0.940   -0.624   -0.241   -0.504   0.063  0.000


In [97]:
Headers = f_Sn.columns.values.tolist()
Indexes = f_Sn.index.values.tolist()
trace = []
num_rows,num_cols = f_Sn.shape
for i in range(num_cols):
  trace.append(go.Scatter(x=f_Sn.index,
                          y = f_Sn[Headers[i]],
                          mode = 'lines', 
                          name = Headers[i])
  )
trace.append(go.Scatter(
    x=f_Sn_exp.index, #columns[:-1],           # x-coordinates of trace
    y=f_Sn_exp['Sn'],          # y-coordinates of trace
    mode='markers',   # scatter mode
    marker = dict(size = 10, color = 'white'),
    error_y=dict(
            type='data', # value of error bar given in data coordinates
            array=f_Sn_exp['Sn_err'],
            visible=True),
    name='exp'
    )
)
fig = go.Figure(data=trace)
fig.update_layout(
    title='',
    width=600,
    height=300,
    margin=dict(l=35, r=35, t=35, b=35),
    xaxis=dict(title='',
               range=[-0.5,4.5],
               tickmode='array',tickvals=[0,1,2,3],
               ticktext=Indexes,
               showline=True,showgrid=True
               ),
    yaxis=dict(title='S<sub>n</sub> [MeV]', 
               range=[-3,4.0]
               ),
    showlegend=True,
    template="plotly_dark",
    legend=dict(
    orientation="h",
    yanchor="bottom",
    y=1.02,
    xanchor="right",
    x=1
  )
)

fig.show()

In [103]:
Headers = f_S2n.columns.values.tolist()
Indexes = f_S2n.index.values.tolist()
trace = []
num_rows,num_cols = f_S2n.shape
for i in range(num_cols):
  trace.append(go.Scatter(x=f_S2n.index,
                          y = f_S2n[Headers[i]],
                          mode = 'lines', 
                          name = Headers[i])
  )
trace.append(go.Scatter(
    x=f_Sn_exp.index, #columns[:-1],           # x-coordinates of trace
    y=f_Sn_exp['S2n'],          # y-coordinates of trace
    mode='markers',   # scatter mode
    marker = dict(size = 10, color = 'white'),
    error_y=dict(
            type='data', # value of error bar given in data coordinates
            array=f_Sn_exp['S2n_err'],
            visible=True),
    name='exp'
    )
)
fig = go.Figure(data=trace)
fig.update_layout(
    title='',
    width=600,
    height=300,
    margin=dict(l=35, r=35, t=35, b=35),
    xaxis=dict(title='',
               range=[-0.5,4.5],
               tickmode='array',tickvals=[0,1,2,3],
               ticktext=Indexes,
               showline=True,showgrid=True
               ),
    yaxis=dict(title='S<sub>n</sub> [MeV]', 
               range=[-3,4.0]
               ),
    showlegend=True,
    template="plotly_dark",
    legend=dict(
    orientation="h",
    yanchor="bottom",
    y=1.02,
    xanchor="right",
    x=1
  )
)

fig.show()